In [1]:
from constants import *
from TwoQubits import *

In [8]:
# define qubits q1 and q2
qq = TwoQubits([0,0,-1],[0,0,-1])

In [9]:
qq.get_matrix()

matrix([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [40]:
class TwoQubits:
    """
    input:
    
    a1, b1:    |psi_1> = a1*|0> + b1*|1>
    a2, b2:    |psi_2> = a2*|0> + b2*|1>
    
    gamma1:  longitudinal relaxation rate 1/T1  (T1: relaxation time)
    gamma2:  transversal relaxation rate 1/T2 = gamma1/1 + gamma_phi (T2: relaxation time, gamma_phi: pure dephasing rate)
    dw:      frequency shift dw = wq - wd (wq: qubit frequency, wd: rotating frame frequency)
    """
    def __init__(self, a1, b1, a2, b2, gamma1, gamma2, dw):
        if (a1**2+b1**2 != 1):
            a1 = np.real(a1/(a1**2+b1**2)) + 1j*np.imag(a1/(a1**2+b1**2))
            b1 = np.real(b1/(a1**2+b1**2)) + 1j*np.imag(b1/(a1**2+b1**2))
            print("Mind that wavefunction |psi_1> = a1*|0> + b1*|1> should be normed a1^2 + b1^2 = 1. "
                  "Parameters were scaled: \na = {} \nb = {}".format(a1, b1))
        if (a2**2+b2**2 != 1):
            a2 = np.real(a2/(a2**2+b2**2)) + 1j*np.imag(a2/(a2**2+b2**2))
            b2 = np.real(b2/(a2**2+b2**2)) + 1j*np.imag(b2/(a2**2+b2**2))
            print("Mind that wavefunction |psi_2> = a2*|0> + b2*|1> should be normed a2^2 + b2^2 = 1. "
                  "Parameters were scaled: \na = {} \nb = {}".format(a2, b2))
        
        c1 = a1*a2
        c2 = a1*b2
        c3 = b1*a2
        c4 = b1*b2
        
        if (c1**2 + c2**2 + c3**2 + c4**2 != 1):
            c1 = c1/(c1**2 + c2**2 + c3**2 + c4**2)
            c2 = c2/(c1**2 + c2**2 + c3**2 + c4**2)
            c3 = c3/(c1**2 + c2**2 + c3**2 + c4**2)
            c4 = c4/(c1**2 + c2**2 + c3**2 + c4**2)
        
        vec = np.matrix([c1,c2,c3,c4]).T
        # a1 a1* a2 a2*   a1 a1* a2 b2*   a1 b1* a2 a2*   a1 b1* a2 b2*
        # a1 a1* b2 a2*   a1 a1* b2 b2*   a1 b1* b2 a2*   a1 b1* b2 b2*
        # b1 a1* a2 a2*   b1 a1* a2 b2*   b1 b1* a2 a2*   b1 b1* a2 b2*
        # b1 a1* b2 a2*   b1 a1* b2 b2*   b1 b1* b2 a2*   b1 b1* b2 b2*
        
        # m11 n11   m11 n12   m12 n11   m12 m12
        # m11 n21   m11 n22   m12 n21   m12 n22
        # m21 n11   m21 n12   m22 n11   m22 n12
        # m21 n21   m21 n22   m11 n21   m22 n22
        
        # a a*  a b* =  m11 m12
        # a* b  b b*    m21 m22
        
        self.matrix = vec*vec.H
        
        self.G1 = gamma1
        self.G2 = gamma2
        self.dw = dw
        
        print(self.matrix)
    
                     
    # ?????
                     
    def propagate(self, t):
        # t: time (in seconds)
        self.matrix = np.matrix([[1+(self.matrix[0,0]-1)*np.exp(-self.G1*t), self.matrix[0,1]*np.exp(1j*self.dw*t)*np.exp(-self.G2*t)],
                  [self.matrix[1,0]*np.exp(-1j*self.dw*t)*np.exp(-self.G2*t), self.matrix[1,1]*np.exp(-self.G1*t)]])
    
    def get_matrix(self):
        return self.matrix
    
    def _set_matrix(self, matrix):
        if type(matrix)!=np.matrix:
            print("Input must be of type numpy.matrix")
        self.matrix = matrix
        
    def get_Bloch_vector(self):
        x = np.real(2*self.matrix[0,1])
        y = -np.imag(2*self.matrix[0,1])
        z = np.real(2*self.matrix[0,0]-1)
        return x,y,z
    

In [41]:
q1 = TwoQubits(1/np.sqrt(2), 1/np.sqrt(2), 1/np.sqrt(2), 1/np.sqrt(2), 2, 3, 0)

Mind that wavefunction |psi_1> = a1*|0> + b1*|1> should be normed a1^2 + b1^2 = 1. Parameters were scaled: 
a = (0.7071067811865476+0j) 
b = (0.7071067811865475+0j)
Mind that wavefunction |psi_2> = a2*|0> + b2*|1> should be normed a2^2 + b2^2 = 1. Parameters were scaled: 
a = (0.7071067811865476+0j) 
b = (0.7071067811865475+0j)
[[0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
 [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
 [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
 [0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]]


In [25]:
import numpy as np

In [27]:
vec1*vec1.H

matrix([[1.+0.j, 0.-1.j, 2.+0.j, 0.+0.j],
        [0.+1.j, 1.+0.j, 0.+2.j, 0.+0.j],
        [2.+0.j, 0.-2.j, 4.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [ ]:
np.matrix([c1,c2,c3,c4])